In [4]:
import pandas as pd
import os
from pathlib import Path
import cv2
from sklearn.model_selection import train_test_split
import shutil
import yaml
from ultralytics import YOLO
import numpy as np
import torch
import torchvision

In [5]:
print("Torch version:", torch.__version__)
print("Torchvision version:", torchvision.__version__)
print("CUDA available:", torch.cuda.is_available())
print("CUDA version:", torch.version.cuda)

Torch version: 2.5.1+cu124
Torchvision version: 0.20.1+cu124
CUDA available: True
CUDA version: 12.4


# Сбор и подготовка данных

In [7]:
labels_path = Path.cwd() / 'data' / 'labels.csv'
ladels_df = pd.read_csv(labels_path)

ladels_df.head()

,image,xmin,ymin,xmax,ymax
0,vid_4_1000.jpg,281.259045,187.035071,327.727931,223.225547
1,vid_4_10000.jpg,15.163531,187.035071,120.329957,236.430180
2,vid_4_10040.jpg,239.192475,176.764801,361.968162,236.430180
3,vid_4_10020.jpg,496.483358,172.363256,630.020260,231.539575
4,vid_4_10060.jpg,16.630970,186.546010,132.558611,238.386422


In [8]:
ladels_df.describe()

,xmin,ymin,xmax,ymax
count,559.000000,559.000000,559.000000,559.000000
mean,263.387486,186.517539,365.324845,231.500472
std,207.368388,9.315974,213.501175,13.987176
min,0.000000,148.399292,28.556446,198.321729
25%,61.387844,180.109294,166.309696,223.225547
50%,236.746744,187.524131,348.585579,230.561454
75%,446.835022,193.881918,571.322721,237.897362
max,645.183792,208.064672,676.000000,308.322072


In [9]:
ladels_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 559 entries, 0 to 558
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   image   559 non-null    object 
 1   xmin    559 non-null    float64
 2   ymin    559 non-null    float64
 3   xmax    559 non-null    float64
 4   ymax    559 non-null    float64
dtypes: float64(4), object(1)
memory usage: 22.0+ KB


In [10]:
ladels_df['bbox'] = ladels_df[['xmin', 'ymin', 'xmax', 'ymax']].values.tolist()
ladels_df = ladels_df.drop(columns=['xmin', 'ymin', 'xmax', 'ymax'])
ladels_df = ladels_df.groupby('image', as_index=False).agg({'bbox': list})

ladels_df.head()

,image,bbox
0,vid_4_1000.jpg,"[[281.2590449, 187.0350708, 327.7279305, 223.2..."
1,vid_4_10000.jpg,"[[15.16353111, 187.0350708, 120.3299566, 236.4..."
2,vid_4_10020.jpg,"[[496.4833575, 172.3632561, 630.0202605, 231.5..."
3,vid_4_10040.jpg,"[[239.1924747, 176.7648005, 361.9681621, 236.4..."
4,vid_4_10060.jpg,"[[16.63096961, 186.5460103, 132.5586107, 238.3..."


In [11]:
len(ladels_df)

355

In [12]:
images_path = Path.cwd() / 'data' / 'images'

images_shapes = {
    'image': os.listdir(images_path),
    'width': [],
    'height': [],
    'channels': []
}

for img_name in os.listdir(images_path):
    img_path = images_path / img_name
    h, w, c = cv2.imread(str(img_path)).shape

    images_shapes['width'].append(w)
    images_shapes['height'].append(h)
    images_shapes['channels'].append(c)

images_df = pd.DataFrame.from_dict(images_shapes)

In [13]:
len(os.listdir(images_path))

1001

In [14]:
images_df.head()

,image,width,height,channels
0,vid_4_19120.jpg,676,380,3
1,vid_4_29940.jpg,676,380,3
2,vid_4_11980.jpg,676,380,3
3,vid_4_6200.jpg,676,380,3
4,vid_4_28880.jpg,676,380,3


In [15]:
images_df.describe()

,width,height,channels
count,1001.0,1001.0,1001.0
mean,676.0,380.0,3.0
std,0.0,0.0,0.0
min,676.0,380.0,3.0
25%,676.0,380.0,3.0
50%,676.0,380.0,3.0
75%,676.0,380.0,3.0
max,676.0,380.0,3.0


In [16]:
images_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   image     1001 non-null   object
 1   width     1001 non-null   int64 
 2   height    1001 non-null   int64 
 3   channels  1001 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 31.4+ KB


In [17]:
images_df['width'].unique(), images_df['height'].unique(), images_df['channels'].unique()

(array([676]), array([380]), array([3]))

In [18]:
total_df = pd.merge(ladels_df, images_df, on='image', how='outer')

In [19]:
total_df.head(10)

,image,bbox,width,height,channels
0,vid_4_1000.jpg,"[[281.2590449, 187.0350708, 327.7279305, 223.2...",676,380,3
1,vid_4_10000.jpg,"[[15.16353111, 187.0350708, 120.3299566, 236.4...",676,380,3
2,vid_4_10020.jpg,"[[496.4833575, 172.3632561, 630.0202605, 231.5...",676,380,3
3,vid_4_10040.jpg,"[[239.1924747, 176.7648005, 361.9681621, 236.4...",676,380,3
4,vid_4_10060.jpg,"[[16.63096961, 186.5460103, 132.5586107, 238.3...",676,380,3
5,vid_4_10080.jpg,NaN,676,380,3
6,vid_4_10100.jpg,"[[447.568741, 160.6258044, 582.0839363, 232.51...",676,380,3
7,vid_4_10120.jpg,"[[168.7554269, 180.6772844, 304.7380608, 246.7...",676,380,3
8,vid_4_10140.jpg,"[[0.0, 188.9913127, 85.11143271, 249.1457529]]",676,380,3
9,vid_4_10160.jpg,NaN,676,380,3


In [20]:
total_df.describe()

,width,height,channels
count,1001.0,1001.0,1001.0
mean,676.0,380.0,3.0
std,0.0,0.0,0.0
min,676.0,380.0,3.0
25%,676.0,380.0,3.0
50%,676.0,380.0,3.0
75%,676.0,380.0,3.0
max,676.0,380.0,3.0


In [21]:
total_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   image     1001 non-null   object
 1   bbox      355 non-null    object
 2   width     1001 non-null   int64 
 3   height    1001 non-null   int64 
 4   channels  1001 non-null   int64 
dtypes: int64(3), object(2)
memory usage: 39.2+ KB


In [22]:
def xyxy_to_yolo(xyxy_bbox, image_width, image_height):
    x_center = ((xyxy_bbox[2] + xyxy_bbox[0]) / 2) / image_width
    y_center = ((xyxy_bbox[3] + xyxy_bbox[1]) / 2) / image_height
    width = (xyxy_bbox[2] - xyxy_bbox[0]) / image_width
    height = (xyxy_bbox[3] - xyxy_bbox[1]) / image_height

    return [x_center, y_center, width, height]


total_df.loc[:, 'bbox'] = total_df.apply(
    lambda row: [xyxy_to_yolo(bbox, row['width'], row['height']) for bbox in row['bbox']]
    if isinstance(row['bbox'], list) else row['bbox'], axis=1
)

In [23]:
total_df.head(10)

,image,bbox,width,height,channels
0,vid_4_1000.jpg,"[[0.45043415340236687, 0.539816602368421, 0.06...",676,380,3
1,vid_4_10000.jpg,"[[0.10021707670857989, 0.5571911197368421, 0.1...",676,380,3
2,vid_4_10020.jpg,"[[0.8332127352071006, 0.5314510939473683, 0.19...",676,380,3
3,vid_4_10040.jpg,"[[0.44464544142011836, 0.5436776061842105, 0.1...",676,380,3
4,vid_4_10060.jpg,"[[0.11034732271449704, 0.5591216215789474, 0.1...",676,380,3
5,vid_4_10080.jpg,NaN,676,380,3
6,vid_4_10100.jpg,"[[0.7615774240384616, 0.517294079868421, 0.198...",676,380,3
7,vid_4_10120.jpg,"[[0.35021707670118346, 0.562339124868421, 0.20...",676,380,3
8,vid_4_10140.jpg,"[[0.06295224312869822, 0.5764961389473685, 0.1...",676,380,3
9,vid_4_10160.jpg,NaN,676,380,3


In [24]:
sizes_to_split = {
    'test_size': 0.2,
    'val_size': 0.15
}

In [25]:
train_val_df, test_df = train_test_split(total_df, test_size=sizes_to_split['test_size'])
train_df, val_df  = train_test_split(train_val_df, test_size=sizes_to_split['val_size'] / (1 - sizes_to_split['test_size']))

In [26]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 201 entries, 500 to 949
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   image     201 non-null    object
 1   bbox      63 non-null     object
 2   width     201 non-null    int64 
 3   height    201 non-null    int64 
 4   channels  201 non-null    int64 
dtypes: int64(3), object(2)
memory usage: 9.4+ KB


In [27]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 650 entries, 80 to 238
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   image     650 non-null    object
 1   bbox      236 non-null    object
 2   width     650 non-null    int64 
 3   height    650 non-null    int64 
 4   channels  650 non-null    int64 
dtypes: int64(3), object(2)
memory usage: 30.5+ KB


In [28]:
val_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 150 entries, 0 to 708
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   image     150 non-null    object
 1   bbox      56 non-null     object
 2   width     150 non-null    int64 
 3   height    150 non-null    int64 
 4   channels  150 non-null    int64 
dtypes: int64(3), object(2)
memory usage: 7.0+ KB


# Создание конфигурационных файлов


In [29]:
def create_folders(data_path):
    if os.path.isdir(data_path):
        shutil.rmtree(data_path)
    for i in ('train', 'val', 'test'):
        for j in ('images', 'labels'):
            os.makedirs(data_path / i / j, exist_ok=True)


def create_config_files(data_path, save_path):
    config = {
        'train': str(Path(data_path) / 'train'),
        'val': str(Path(data_path) / 'val'),
        'test': str(Path(data_path) / 'test'),
        'nc': 1,
        'names': {0: 'Car'}
        }

    config_path = save_path / 'data_config.yaml'
    with open(config_path, "w") as file:
        yaml.dump(config, file, default_flow_style=False)


    return str(config_path)


def create_dataset_files(df, save_path, images_path):
    for _, row in df.iterrows():
        img_name = row['image']
        if isinstance(row['bbox'], list):
            with open(save_path / 'labels' / img_name.replace('jpg', 'txt'), 'w') as file:
                for box in row['bbox']:
                    file.write(' '.join(map(str, [0] + box)) + '\n')
        shutil.copyfile(str(images_path / img_name), save_path / 'images' / img_name)

In [30]:
images_path = Path.cwd() / 'data' / 'images'
split_data_path = Path.cwd() / 'dataset'

create_folders(split_data_path)

config_path = create_config_files(split_data_path, Path.cwd())

create_dataset_files(train_df, split_data_path / 'train', images_path)
create_dataset_files(val_df, split_data_path / 'val', images_path)
create_dataset_files(test_df, split_data_path / 'test', images_path)

# Обучение

In [31]:
model = YOLO('yolo11n.pt')

100%|██████████| 5.35M/5.35M [00:00<00:00, 97.5MB/s]


In [32]:
EPOCH = 30
IMG_SIZE = 640
BATCH_SIZE = 16

In [33]:
result = model.train(
    data=config_path,
    epochs=EPOCH,
    imgsz=IMG_SIZE,
    batch=BATCH_SIZE,
    patience=False,
    device=0,
    single_cls=True,
    plots=True,
    augment=True,
    pretrained=True,
    cache=True,
)

Ultralytics 8.3.74 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=/content/data_config.yaml, epochs=30, time=None, patience=False, batch=16, imgsz=640, save=True, save_period=-1, cache=True, device=0, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=True, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True

100%|██████████| 755k/755k [00:00<00:00, 21.1MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      6640  ultralytics.nn.modules.block.C3k2            [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
  4                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  6                  -1  1     87040  ultralytics.nn.modules.block.C3k2            [128, 128, 1, True]           
  7                  -1  1    295424  ultralytics

train: Scanning /content/dataset/train/labels... 236 images, 414 backgrounds, 0 corrupt: 100%|██████████| 650/650 [00:00<00:00, 3378.82it/s]


train: New cache created: /content/dataset/train/labels.cache
WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


...

      30/30      2.38G      1.037     0.5491      1.069          6        640: 100%|██████████| 41/41 [00:12<00:00,  3.39it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:00<00:00,  5.45it/s]

                   all        150        103      0.951      0.981      0.979      0.641



30 epochs completed in 0.117 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 5.4MB
Optimizer stripped from runs/detect/train/weights/best.pt, 5.4MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.74 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 238 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:02<00:00,  2.42it/s]


                   all        150        103      0.968      0.951      0.985      0.644
Speed: 0.1ms preprocess, 9.3ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to runs/detect/train


In [34]:
result

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7e6404192a90>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

# Метрики

In [37]:
model_path = Path.cwd() / 'runs/detect/train/weights/best.pt'
model = YOLO(model_path)

In [38]:
metrics = model.val(
    data='/content/data_config.yaml',
    imgsz=IMG_SIZE,
    batch=BATCH_SIZE,
    conf=0.5,
    iou=0.5,
    device=0,
    plots=True,
    split='test'
    )

Ultralytics 8.3.74 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 238 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


val: Scanning /content/dataset/test/labels... 63 images, 138 backgrounds, 0 corrupt: 100%|██████████| 201/201 [00:00<00:00, 3714.86it/s]

val: New cache created: /content/dataset/test/labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:02<00:00,  6.09it/s]


                   all        201         88      0.977      0.966      0.975      0.703
Speed: 1.5ms preprocess, 4.4ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to runs/detect/val


In [39]:
metrics

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7e6404e8f990>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

# Экспорт

In [40]:
model.export(
    format='onnx',
    imgsz=IMG_SIZE,
    half=False,
    simplify=True,
    dynamic=False,
    device=0,
    )

Ultralytics 8.3.74 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)

PyTorch: starting from '/content/runs/detect/train/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 5, 8400) (5.2 MB)
requirements: Ultralytics requirements ['onnx>=1.12.0', 'onnxslim', 'onnxruntime-gpu'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 178.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.9/142.9 kB 220.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.5/291.5 MB 145.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 184.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 215.0 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 20.3s, installed 3 packages: ['onnx>=1.12.0', 'onnxslim', 'onnxruntime-gpu']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect


ONNX: starting export with onnx 1.17.0 opset 19..

'/content/runs/detect/train/weights/best.onnx'